In [6]:
import pandas as pd
from mysql_connection import *
from transform_API import get_categories
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import requests
import os
from dotenv import load_dotenv 
import sys
import random
from mysql_connection import *
import warnings
warnings.filterwarnings('ignore')

In [7]:
def yelp_ER():
    
    """
        Esta funcion realiza el proceso de ETL completo respecto de la API de yelp para los restaurantes en la base datos mysql.
        Para esto aplica las funciones:
            * extract_businesses
            * transform_business
            * get_table
            * mysql_get_connection
            * get_categories
    
    """
    
    yelp_new_data = pd.read_parquet(r'datalake\business_trasnform.parquet') # Realizo las trasnformaciones necesarias para que los datos esten limpios

    yelp_origen = get_table('business_yelp') # Cargo de la base de datos la tabla de yelp en un dataframe
    
    yelp_new_data = yelp_new_data[~(yelp_new_data['business_id'].isin(yelp_origen['business_id']))] #De los restaurantes extraidos tomo solo los que su id NO esta en la DB
    print(f'La cantidad de restaurantes a ingestar es {yelp_new_data.shape[0]}')
    if yelp_new_data.shape[0] != 0:
        conexion = get_connection_mysql() # Genero una conexion a mysql
        cursor = conexion.cursor() 
        
        consulta = "INSERT INTO business_yelp  VALUES(%s,%s,%s,%s,%s,%s)" 
        yelp_insert = yelp_new_data[['business_id','name','latitude','longitude','stars','state_id']].copy()
        cursor.executemany(consulta,yelp_insert.values.tolist() ) # Inserto los nuevos locales, sin insertar las categorias
        print(f'{yelp_insert.shape[0]} nuevos negocios')
        conexion.commit()
        conexion.close()
        
        categories_origen = get_table('categories') # Cargo la tabla de categorias de la base de datos.    
        max_id = categories_origen['categories_id'].max()
        categorias_new_data = get_categories(yelp_new_data.copy())

        #Agrego la categoria Restaurants a cada local
        
        df_restaurant = categorias_new_data.drop_duplicates(subset='business_id').copy()
        df_restaurants = categorias_new_data.drop_duplicates(subset='business_id').copy()
        
        df_restaurant['categories'] = 'restaurants'
        df_restaurants['categories'] = 'restaurants'
        categorias_new_data = pd.concat([categorias_new_data,df_restaurant])
        categorias_new_data = pd.concat([categorias_new_data,df_restaurants])

        
        
        categorias_new = categorias_new_data[~(categorias_new_data['categories'].isin(categories_origen['name']))] # Selecciono las categorias que no estan en la DB
        categorias_new.loc[:, 'categories_id'] = range(max_id + 1, max_id + 1 + categorias_new.shape[0])


        categories = categorias_new.drop_duplicates(subset='categories').copy() # Elimino las categorias duplicadas y las convierto en lista de listas.
        conexion = get_connection_mysql() 
        cursor = conexion.cursor()
        
        # Ingesto las nuevas categorias.
        consulta = "INSERT INTO categories  VALUES(%s,%s)"
        cursor.executemany(consulta, categories[['categories_id','categories']].values.tolist())
        print(f'{categories.shape[0]} nuevas categorias ingestadas')
        conexion.commit()
        conexion.close()
        
        
        categories_acualizada = get_table('categories') # Cargo la tabla de categorias actualizada.
        
        #Hago un join entre la tabla business_id,categoria creada anteriormente con las categorias de la BD, y me quedo solo con business_id y categoria id
        categorias_yelp_new =  pd.merge(categories_acualizada,categorias_new_data,left_on='name',right_on='categories',how='inner')
        
        conexion = get_connection_mysql()
        
        categorias_yelp_new['categories_id'] = categorias_yelp_new['categories_id'].astype(int)
        
        print(categorias_yelp_new)
        # Como business id ya es unico simplemente agrego las filas a la tabla cateogires_yelp
        try:
            cursor = conexion.cursor()
            consulta = "INSERT INTO categories_yelp  VALUES(%s,%s)"
            cursor.executemany(consulta, categorias_yelp_new[['business_id','categories_id']].values.tolist())
            conexion.commit()
            conexion.close()
        except Exception as e:
            print(f"Error al ejecutar la consulta SQL: {e}")
            # Aquí puedes agregar código adicional para manejar la excepción según tus necesidades.
            # Por ejemplo, podrías hacer un rollback si es necesario.
        finally:
            # Este bloque se ejecutará siempre, asegurando que la conexión se cierre incluso en caso de excepción.
            if conexion and conexion.open:
                conexion.rollback()  # Hacer un rollback en caso de excepción antes de cerrar la conexión.
                conexion.close()
    else:
        return 'No habian restaurantes para ingestar'



    

    



In [ ]:
yelp_ER()

In [ ]:
#------------------------------------------------------------------------      
#----------------------------REVIEWS-------------------------------------    
#------------------------------------------------------------------------ 

In [ ]:

def yelp_review_ER():
    """
    Esta función realiza el proceso de ETL completo respecto de la API de Yelp para las reviews de restaurantes y las sube en la base de datos MySQL.
    Para esto aplica las funciones:
        * extract_businesses
        * transform_business
        * get_table
        * mysql_get_connection
    """
    # Leer datos de reviews desde el archivo parquet
    
    # Obtener las reviews existentes en la base de datos
    
    review_new_data = pd.read_parquet(r'datalake\reviews_yelp_transform.parquet') # Hago las trasnformaciones sobre el dataframe.
    
    reviews_yelp_origen = get_review_yelp('reviews_yelp') # Consulto la tabla las ultimas resenas

    users_old = get_filtered_table('user_yelp', review_new_data['user_id'].unique().tolist())
    
    #Filtro solo las reviews donde su columna date sea mayor a la maxima existente en la base de datos.
    print(f'{review_new_data.shape[0]} reviews a ingestar')
    
    review_new_data = pd.merge(review_new_data, reviews_yelp_origen[['review_id']], on='review_id', how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)
    
    #review_new_data = review_new_data[(~review_new_data['review_id'].isin(reviews_yelp_origen['review_id']))]
    review_new_data['date'] = pd.to_datetime(review_new_data['date'])


    
    #### USERS ####  

    grouped_data = review_new_data.groupby('user_id') # Rrealizo el agrupamiento por usuarios y calculas las metricas.
    min_dates = grouped_data['date'].min()
    first_names = grouped_data['name'].first()
    review_counts = grouped_data['review_id'].count()
    mean_stars = grouped_data['stars'].mean()
    users = pd.DataFrame({
        'name': first_names,
        'creation': min_dates,
        'review_count': review_counts,
        'stars': mean_stars
    }).reset_index()
    users['influence'] = 0
    users = users[['user_id','name','creation','review_count','influence','stars']]
    #### DATAFRAME CON LOS USUARIOS NUEVOS
    new_users = users[~(users['user_id'].isin(users_old['user_id']))]
    new_users['influence'] = 0
    
    print(users.head())
    
     #### DATAFRAME CON LOS USUARIOS EXISTENTES
    #exist_user = users[(users['user_id'].isin(users_old['user_id']))] # Usuarios existentes
    
    
    exist_user = pd.concat([users_old, users[users['user_id'].isin(users_old['user_id'])]]) # Hago un merge de los usuarios de la BD y la llegada
    # Para los nuevos usuarios encuentro la nueva review_count y stars
    exist_user['stars'] = exist_user['stars'].astype('float')
    # Realizo el agrupamiento por usuarios existentes y calculo las metricas.
    grouped_data = exist_user.groupby('user_id')
    min_dates = grouped_data['creation'].min()
    first_names = grouped_data['name'].first()
    review_counts = grouped_data['review_count'].count()
    mean_stars = grouped_data['stars'].mean()
    exist_user = pd.DataFrame({
        'name': first_names,
        'creation': min_dates,
        'review_count': review_counts,
        'stars': mean_stars
    }).reset_index()
    
    if not exist_user.empty:
        try:
            conexion = get_connection_mysql()
            cursor = conexion.cursor()
            
            consulta_new_user = (
            'INSERT INTO user_yelp  VALUES(%s,%s,%s,%s,%s,%s)'
                )

            # Ejecutar la consulta con execumany.
            cursor.executemany(consulta_new_user, new_users[['user_id', 'name','creation', 'review_count', 'influence','stars']].values.tolist())
            print(f'{new_users.shape[0]} usuarios nuevos cargados.')
            # Consulta de actualización con placeholders.
            consulta_old_user = (
                "UPDATE user_yelp "
                "SET name = %s, creation = %s, review_count = %s, stars = %s "
                "WHERE user_id = %s"
            )

            # Ejecutar la consulta con execumany.
            cursor.executemany(consulta_old_user, exist_user[['name', 'creation', 'review_count', 'stars', 'user_id']].values.tolist())
            print(f'{exist_user.shape[0]} usuarios actualizados.')
            conexion.commit()
        except Exception as e:
            print(f"Error al ejecutar la consulta SQL: {e}")
            # Maneja la excepción según tus necesidades.
            # Puedes hacer un rollback si es necesario.
        finally:
            # Cierra la conexión en el bloque finally para asegurar que se cierre incluso en caso de excepción.
            if conexion and conexion.open:
                conexion.rollback()
                conexion.close()

    
    # Verificación de review_id antes de la inserción
    conexion = get_connection_mysql()
    cursor = conexion.cursor()

    review_id_list = review_new_data['review_id'].tolist()

    # Crear una cadena con los review_id para la consulta SQL
    review_id_str = ','.join([f"'{review_id}'" for review_id in review_id_list])

    # Consulta SQL para verificar si los review_id existen en la tabla
    consulta_existencia = f"SELECT review_id FROM reviews_yelp WHERE review_id IN ({review_id_str})"
    cursor.execute(consulta_existencia)

    # Obtener los review_id que existen en la base de datos
    review_id_existente = [row[0] for row in cursor.fetchall()]

    # Filtrar la lista original de review_id para obtener aquellos que no existen en la base de datos
    review_id_no_existente = [review_id for review_id in review_id_list if review_id not in review_id_existente]

    conexion.close()

    # Filtrar el DataFrame para mantener solo los registros con review_id que no existen en la base de datos
    review_new_data_filtered = review_new_data[review_new_data['review_id'].isin(review_id_no_existente)]

    # Inserción de registros en la tabla reviews_yelp para los review_id que no existen
    try:
        conexion = get_connection_mysql()
        cursor = conexion.cursor()

        review_new_data_filtered.drop_duplicates(subset='review_id', inplace=True)

        consulta = "INSERT INTO reviews_yelp VALUES(%s, %s, %s, %s, %s, %s)"
        cursor.executemany(consulta, review_new_data_filtered.drop(columns=['name']).values.tolist())

        
        
        print(f'{review_new_data_filtered.shape[0]} nuevas reviews insertadas')
        print(review_new_data_filtered.head())
        conexion.commit()
        
        return review_new_data_filtered
    except Exception as e:
        print(f"Error al ejecutar la consulta SQL: {e}")
        if conexion:
            conexion.rollback()
    finally:
        if conexion:
            conexion.close()

            

In [ ]:
yelp_review_ER()

# Api yelp

In [9]:

api_key_yelp =  "5DIPr_h-AqFcwZuvZboWCR110mmQ6lXWLHYZKpkIiL_QKqguUurch2AwJwkmDk3ywshaUd_MXkYFf0-MG7i1hEd1hIi_8fSGeRU5M0d_Nuc4QwC7NP-rioH0O-lpZXYx" # Cargo la variable de entorno


# Funcion que consulta la API de yelp para obtener los locales por estado.    
def get_business_API(state):
    """Esta funcion realiza una consulta a la API de yelp para obtener los restaurantes por estado.

    Args:
        state (string): Codigo de estado (Texas=TX) del estado requerido. 

    Returns:
        pd.DataFrame: Data frame de la información de un restaurant en concreto.
    """
    
    
    url = f'https://api.yelp.com/v3/businesses/search'

    params = {
        'location': state,
        'categories':','.join(['restaurant','Restaurant','restaurants','Restaurants']),
        'limit':50
    }

    headers = {
        'Authorization': f'Bearer {api_key_yelp}',
        'accept': 'application/json'
    }

    response = requests.get(url, headers=headers,params=params)

    if response.status_code == 200:
        data = response.json()
        # Convierte el JSON en un DataFrame de pandas
        #df = pd.json_normalize(data)
        businesses = pd.json_normalize(data['businesses'])
        return businesses
    else:
        print(f'Error en la solicitud. Código de estado: {response.status_code}')
        print(response.text)

In [10]:
# Funcion que carga desde la API hasta  50 datos de cada estado.        
def extract_businesses():
    
    """ Esta funcion aplica la funcion business para 4 estados seleccionados.

    Returns:
        pd.DataFrame: Concatenacion de los restaurantes de los 4 estados.
    """
    
    
    url = f'https://api.yelp.com/v3/businesses/search'

    yelp_bussines = pd.DataFrame()
    for state in ['CA','FL','NJ','IL']:
        businesses = get_business_API(state)
        yelp_bussines = pd.concat([businesses,yelp_bussines])
    yelp_bussines.to_parquet(r'datalake\business_API.parquet')
    return yelp_bussines


In [11]:
business = extract_businesses()

In [12]:
# Funcion que apartir de la id de un local extrae las reviews
def reviews_yelp_API(business_id):
    
    """ Esta funcion retorna para un business_id la información de reviews en un DataFrame.

    Returns:
        pd.DataFrame: Data Frame de reviews.
    """
    
    
    url =f'https://api.yelp.com/v3/businesses/{business_id}/reviews?limit=50&sort_by=newest'

    headers = {
        'Authorization': f'Bearer {api_key_yelp}',
        'accept': 'application/json'
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        reviews_list = data.get('reviews', [])
        df = pd.json_normalize(reviews_list)
        return  df

    else:
        print(f'Error en la solicitud. Código de estado: {response.status_code}')
        

In [13]:
# Funcion que a partir de los locales ya existentes en DW extrae las reseñas.
def get_reviewsYelp_API():
    
    """ Esta funcion aplica la funcion reviews_yelp_api  para todos los business que estan en la base de datos aplicando get_table(yelp),
    ademas lo hace 496 para tener en cuenta solo las peticione restantes,
    Falta ver como modificar el business_id de input.

    Returns:
        pd.DataFrame: DataFrame de las revies.
    """
    
    
    yelp = get_table('business_yelp') # Obtengo la tabla de restaurantes de la DB
    business_ids_distinct_list = yelp.dropna(subset='business_id')['business_id'].unique().tolist()
    random.shuffle(business_ids_distinct_list)
    reviews_business = pd.DataFrame()
    iter = 0
    for business_id in business_ids_distinct_list:
        if business_id is None: continue
        if iter <= 10:
            iter += 1
            reviews = reviews_yelp_API(business_id)
            reviews['business_id'] = business_id
            reviews_business = pd.concat([reviews,reviews_business])
            reviews['time_created'] = pd.to_datetime(reviews['time_created'])
            reviews = reviews[reviews['time_created'].dt.year >= 2015]
            
        else :
            reviews['time_created'] = pd.to_datetime(reviews['time_created'])
            reviews = reviews[reviews['time_created'].dt.year >= 2015]
            reviews_business.to_parquet(r'datalake\reviews_yelp.parquet')
            return reviews_business
    reviews['time_created'] = pd.to_datetime(reviews['time_created'])
    reviews = reviews[reviews['time_created'].dt.year >= 2015]
    reviews_business.to_parquet(r'datalake\reviews_yelp.parquet')
    return reviews_business


In [14]:
reviews = get_reviewsYelp_API()
reviews

,id,url,text,rating,time_created,user.id,user.profile_url,user.image_url,user.name,business_id
0,aoNnLo8XZHhhhG4IByCanQ,https://www.yelp.com/biz/pholicious-brandon?ad...,I read every single review before trying this ...,5,2019-05-29 11:13:44,9PoFj-oL-ll2lUqdny66EQ,https://www.yelp.com/user_details?userid=9PoFj...,https://s3-media2.fl.yelpcdn.com/photo/m0doJi8...,Lauren M.,xK7Ie1utDh9IW7tQfkLpZA
1,eUkXOxMOC31CAr4vyomeZA,https://www.yelp.com/biz/pholicious-brandon?ad...,The spring rolls were not good in the least. W...,2,2020-02-23 13:12:14,6XyQ3fGGMCVS1W7ma9chng,https://www.yelp.com/user_details?userid=6XyQ3...,https://s3-media3.fl.yelpcdn.com/photo/Cm14YAF...,Nate T.,xK7Ie1utDh9IW7tQfkLpZA
2,SnD0YY5dph3TW5w5-QoJWw,https://www.yelp.com/biz/pholicious-brandon?ad...,"Probably the healthiest choice for lunch, or d...",3,2018-07-07 20:12:16,Za2N7jB06TaW3xnNt7Hjpg,https://www.yelp.com/user_details?userid=Za2N7...,https://s3-media1.fl.yelpcdn.com/photo/rrt7M9T...,Tony N.,xK7Ie1utDh9IW7tQfkLpZA
0,gcyll2rabGCg4O2rfOevSw,https://www.yelp.com/biz/little-caesars-pizza-...,"Criticisms: Yes, this is pure mass-produced co...",3,2010-01-05 00:31:14,hTDIebPoUXeNdyorFlUkQw,https://www.yelp.com/user_details?userid=hTDIe...,https://s3-media2.fl.yelpcdn.com/photo/s1lW2MC...,Edward M.,pUeTnGsnVi-H8RJ6prFFgw
1,6BstLC9Xya07hhra-rgDvA,https://www.yelp.com/biz/little-caesars-pizza-...,This is by far Santa Barbara's best restaurant...,5,2022-12-22 14:07:36,JG3XzRsauHL5cy2jhjL80w,https://www.yelp.com/user_details?userid=JG3Xz...,https://s3-media3.fl.yelpcdn.com/photo/zq3DPT2...,Sebastion R.,pUeTnGsnVi-H8RJ6prFFgw
2,QskjqhGMyZWa23wAGa7rsw,https://www.yelp.com/biz/little-caesars-pizza-...,Whenever I order food to get delivered from he...,1,2023-08-07 07:51:23,NR-8YBoc6YdhXHNBjokafw,https://www.yelp.com/user_details?userid=NR-8Y...,https://s3-media1.fl.yelpcdn.com/photo/b_wxS1r...,Nicholas G.,pUeTnGsnVi-H8RJ6prFFgw
0,9JEjjfp0hyH6tuqA5CDB5Q,https://www.yelp.com/biz/chipotle-mexican-gril...,"The food was good, some of the food had been s...",3,2023-10-08 16:41:31,-pXRpfqsg8r1BRr4Bdk78w,https://www.yelp.com/user_details?userid=-pXRp...,None,Fred K.,W4EklwiIeQNH6QhkcGllQQ
1,uElhUASq5K3LCP-Myp3AYw,https://www.yelp.com/biz/chipotle-mexican-gril...,This place is always super busy. Even had an i...,2,2023-04-27 07:09:54,FuQhthi1rfpFc4FR7YZTLQ,https://www.yelp.com/user_details?userid=FuQht...,https://s3-media2.fl.yelpcdn.com/photo/sP_cvnV...,Adrian F.,W4EklwiIeQNH6QhkcGllQQ
2,sHiYmEV8i3XE-NuiK-HeDg,https://www.yelp.com/biz/chipotle-mexican-gril...,Was an average trip to Chipotle. The workers w...,3,2022-05-17 10:22:24,uelWw7iVJW-L0YhuuqPo1g,https://www.yelp.com/user_details?userid=uelWw...,https://s3-media4.fl.yelpcdn.com/photo/eSD6haq...,Sonji C.,W4EklwiIeQNH6QhkcGllQQ
0,Mg5RIVS_4MpdrJeZ0k4hyQ,https://www.yelp.com/biz/samaria-cafe-tampa?ad...,I tried out a new (to me) cafe in Downtown Tam...,5,2023-11-16 14:14:29,ZDqYBzL1_rp2LE_amVXsKw,https://www.yelp.com/user_details?userid=ZDqYB...,https://s3-media3.fl.yelpcdn.com/photo/RWmTHCx...,PJ R.,gcLHVHLDOozjllD-yiAnlQ


In [ ]:
reviews